In [1]:
import numpy as np
import numba as nb
import time 

In [28]:
def get_rownums(stidx,endidx,spec_arr):
    l=np.searchsorted(spec_arr,stidx,side='left')
    r=np.searchsorted(spec_arr,endidx,side='left')
    if(spec_arr[r]>endidx):
        r=r-1
    return l, r

def get_rows_from_specnum(spec1,spec2, spec_num, spectra_per_packet):

#     print(f"received spec1 = {spec1} and spec2 = {spec2}")
    l = np.searchsorted(spec_num,spec1,side='left')
    r = np.searchsorted(spec_num,spec2,side='left')
    # print(f"spec1={spec1} spec2={spec2}")
    # print(f"spec_num len {spec_num.shape[0]}, last spec_num {spec_num[-1]}, l={l}, r={r}")
    diff1=-1
    diff2=-1
    if(l<spec_num.shape[0]):
        diff1 = spec1-spec_num[l]
    if(r<spec_num.shape[0]):
        diff2 = spec2-spec_num[r] # will give distance from right element in not equal cases

    if(diff1!=0 and diff2!=0):
        diff1 = spec1-spec_num[l-1]
        diff2 = spec2-spec_num[r-1]
        if(diff1<spectra_per_packet):
            idx1=(l-1)*spectra_per_packet+diff1
        else:
            idx1=l*spectra_per_packet
        if(diff2<spectra_per_packet):
            idx2=(r-1)*spectra_per_packet+diff2
        else:
            idx2=r*spectra_per_packet
    elif(diff1==0 and diff2!=0):
        idx1 = l*spectra_per_packet
        diff2 = spec2-spec_num[r-1]
        if(diff2<spectra_per_packet):
            idx2=(r-1)*spectra_per_packet+diff2
        else:
            idx2=r*spectra_per_packet
    elif(diff1!=0 and diff2==0):
        idx2 = r*spectra_per_packet
        diff1 = spec1-spec_num[l-1]
        if(diff1<spectra_per_packet):
            idx1=(l-1)*spectra_per_packet+diff1
        else:
            idx1=l*spectra_per_packet
    else:
        idx1 = l*spectra_per_packet
        idx2 = r*spectra_per_packet
    return int(idx1),int(idx2)

In [3]:
spec_num = np.loadtxt('../spec_idx.txt',dtype='int')
missing_loc = np.loadtxt('../missing_loc.txt',dtype='int')
missing_num = np.loadtxt('../missing_num.txt',dtype='int')

In [4]:
len(spec_num)*5

2285190

In [5]:
@nb.njit(parallel=True)
def fill_arr(myarr,specnum, spec_per_packet=5):
    
    n=len(specnum)
    for i in nb.prange(n):
        for j in nb.prange(spec_per_packet):
            myarr[i*spec_per_packet+j] = specnum[i]+j

# @nb.njit()
# def fill_arr2(myarr,specnum, spec_per_packet=5):
# 	for i in nb.range(myarr.shape[0]):
# 		m=i//spec_per_packet
# 		n=i%spec_per_packet
# 		myarr[i]=spec_num[m]+n
    

In [29]:
get_rownums(23,2411215, filledarr)

IndexError: index 2285190 is out of bounds for axis 0 with size 2285190

In [18]:
missing_loc

array([    255,  125940,  177820,  178305,  301135,  301890,  311925,
        318970,  326085,  333130,  373675,  389580,  442960,  473815,
        550375,  579485,  611530,  620330,  652415,  662685,  668245,
        703130,  713155,  720150,  919350,  940750,  960825,  980175,
       1089685, 1089790, 1090045, 1103730, 1656860, 1691220, 1759545,
       1812225, 1827450, 1861245, 1896475, 1931190, 1938510, 2042475,
       2051615, 2072795, 2095335, 2117720, 2244455, 2245510, 2246420,
       2289660, 2289830, 2290085, 2290225, 2316445, 2333250, 2333335,
       2333470, 2333610, 2333705])

In [31]:
missing_num

array([106805,     40,    405,    270,    595,     35,     85,     75,
           85,     65,     65,    500,    135,     80,     70,     45,
           80,    245,    590,     40,   2485,     25,     35,     80,
          600,    555,    550,    150,     25,    175,     85,     10,
          280,     85,   5320,    185,    435,    350,    315,    280,
          525,     20,     60,     60,    385,     95,   1045,    840,
           40,     90,    175,     60,    140,      5,      5,     55,
           60,     15,     15])

In [43]:
np.searchsorted(filledarr,107060,side='left')

255

In [40]:
255+106805

107060

In [36]:
filledarr[19134:19136]

array([125939, 125980])

In [42]:
filledarr[255]

107060

In [7]:
filledarr = np.zeros(len(spec_num)*5,dtype='int64')
# filledarr2 = np.zeros(len(spec_num)*5,dtype='int64')

In [8]:
f1=fill_arr(filledarr,spec_num)
# f2=fill_arr2(filledarr2,spec_num)

In [10]:
24

array([      0,       5,      10, ..., 2411200, 2411205, 2411210])

In [13]:
filledarr2

array([      0,       1,       2, ..., 2411212, 2411213, 2411214])

In [15]:
%timeit fill_arr(filledarr,spec_num)


439 µs ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [104]:
np.savetxt('../filledarr.txt',filledarr)

In [37]:
np.searchsorted(filledarr,2411214,side='left')

2285189

In [34]:
filledarr[-10:]

array([2411205, 2411206, 2411207, 2411208, 2411209, 2411210, 2411211,
       2411212, 2411213, 2411214], dtype=uint32)

In [36]:
len(spec_num)*5

2285190

In [100]:
filledarr.shape

(2285190,)

In [54]:
%timeit get_rownums(260,400000,filledarr)

2.35 ms ± 495 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%timeit get_rows_from_specnum(260,400000,spec_num,5)

4.72 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [103]:
# arr = np.random.randint(0,10000000, size=3000000)
# arr.sort()
print(arr.shape)
niter=20
t1=time.time()
for i in range(niter):
    l,r=get_rownums(250,400000,arr)
t2=time.time()
print((t2-t1)*1e6/niter)



(2411214,)
6.985664367675781


In [58]:
def get_common_rows(missing_blocks1,missing_num1, missing_blocks2,missing_num2, acclen):

    st=0
    b1=len(missing_blocks1)
    b2=len(missing_blocks2)
    # print("Block lens", b1, b2)
    if(len(missing_blocks1)>0):
        en=missing_blocks1[0][0]
    else:
        en=acclen
    if(b2==0):
        return np.arange(0,acclen-missing_num1)
    j=0
    i=0
    cum_missing=0
    tot=0
    ids=[]
    while(i<b1 and j<b2):
        en=missing_blocks1[i][0]
        # print("en is",en)
        if(missing_blocks2[j][0]<=en and missing_blocks2[j][1]>st):
            if(missing_blocks2[j][0]>st):
                if(missing_blocks2[j][1]>=en):
                    print("flag 1")
                    ids.append([st-cum_missing,missing_blocks2[j][0]-cum_missing])
                    tot+=(missing_blocks2[j][0]-st)
                    st=missing_blocks1[i][1]
                    cum_missing+=(missing_blocks1[i][1]-missing_blocks1[i][0])
                    i+=1
                else:
                    # print("flag 2")
                    ids.append([st-cum_missing,missing_blocks2[j][0]-cum_missing])
                    tot+=(missing_blocks2[j][0]-st)
                    st=missing_blocks2[j][1]
                    j+=1
            else:
                if(missing_blocks2[j][1]>=en):
                    # print("flag 3")
                    st=missing_blocks1[i][1]
                    cum_missing+=(missing_blocks1[i][1]-missing_blocks1[i][0])
                    i+=1 #whole present block1 is useless.
                else:
                    # print("flag 4")
                    st=missing_blocks2[j][1]
                    j+=1
        else:
            ids.append([st-cum_missing,en-cum_missing])
            tot+=(en-st)
            st=missing_blocks1[i][1]
            cum_missing+=(missing_blocks1[i][1]-missing_blocks1[i][0])
            # print("flag 5")
            i+=1
    # print("PHASE 1: st is",st,"and end is",en,"cum missing is",cum_missing, "i and j", i,j)
    # print("PHASE 1 ids",ids)
    
    en=acclen
    while(j<b2):
        if(missing_blocks2[j][0]<en and missing_blocks2[j][1]>st):
            if(missing_blocks2[j][0]>st):
                if(missing_blocks2[j][1]==en): #can't be greater anymore. looking at last present block
                    # print("flag 1")
                    ids.append([st-cum_missing,missing_blocks2[j][0]-cum_missing])
                    tot+=(missing_blocks2[j][0]-st)
                else:
                    # print("flag 2")
                    ids.append([st-cum_missing,missing_blocks2[j][0]-cum_missing])
                    tot+=(missing_blocks2[j][0]-st)
                    st=missing_blocks2[j][1]
                    if(j==(b2-1)):
                        ids.append([st-cum_missing,acclen-cum_missing])
            else:
                if(missing_blocks2[j][1]<en):
                    # print("flag 4")
                    st=missing_blocks2[j][1]
                    if(j==(b2-1)):
                        ids.append([st-cum_missing,acclen-cum_missing])
                        tot+=(acclen-st)
                    # print("here 2")
        else:
            ids.append([st-cum_missing,acclen-cum_missing]) 
            tot+=(acclen-st)
        
        j+=1
    while(i<b1):
        # print("i again")
        en=missing_blocks1[i][0]
        ids.append([st-cum_missing,en-cum_missing])
        tot+=(en-st)
        st=missing_blocks1[i][1]
        cum_missing+=(missing_blocks1[i][1]-missing_blocks1[i][0])
        if(i==(b1-1)):
            ids.append([missing_blocks1[i][1]-cum_missing,acclen-cum_missing])
        i+=1
    # print("DONE")
    arr=np.zeros(tot)
    n=0
    # print(ids)
    print(ids)
    for i,b in enumerate(ids):
        arr[n:n+(b[1]-b[0])]=np.arange(b[0],b[1])
        n=b[1]-b[0]
    return arr,tot

In [60]:
missing_block1 = [[40,60],[80,90]]
missing_block2 = [[50,85]]
missingnum1 = 30
missingnum2 = 35
acclen=100

get_common_rows(missing_block2,missingnum2,missing_block1,missingnum1,acclen)

flag 1
[[0, 40], [50, 65], [55, 65]]


(array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 25.,
        26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
        39., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61.,
        62., 63., 64.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 65)

In [45]:
acclen=100
missingnum1=40
missingnum2=30
arr1=np.zeros(acclen-missingnum1)
arr2=np.zeros(acclen-missingnum2)
ids1=[[0,20],[40,70],[90,100]]
ids2=[[0,30],[60,100]]
# ids1=[[20,40],[60,100]]
# ids2=[[0,10],[50,80]]
n=0
print(arr1.shape)
for i,b in enumerate(ids1):
        arr1[n:n+(b[1]-b[0])]=np.arange(b[0],b[1])
        n+=b[1]-b[0]
n=0
for i,b in enumerate(ids2):
        arr2[n:n+(b[1]-b[0])]=np.arange(b[0],b[1])
        n+=b[1]-b[0]
print(arr1,arr2)

(60,)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54. 55.
 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 90. 91. 92. 93.
 94. 95. 96. 97. 98. 99.] [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 60. 61. 62. 63. 64. 65.
 66. 67. 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83.
 84. 85. 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]


In [48]:
def get_xcorr_rows(spec1,spec2):
    idx1=[]
    idx2=[]
    i=0;j=0 
    while(i<len(spec1) and j<len(spec2)):
        if(spec1[i]==spec2[j]):
            idx1.append(i)
            idx2.append(j)
            i+=1
            j+=1
        elif(spec1[i]>spec2[j]):
            j+=1
        else:
            i+=1
    return np.asarray(idx1),np.asarray(idx2)

In [55]:
def test():

    i=0;j=0 
    while(j<400000):
        if(j%2==0):
            i+=1
        j+=1


In [57]:
%timeit test()

24.2 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
id1,id2=get_xcorr_rows(arr1,arr2)

In [50]:
print(id1,id2)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 30 31 32 33
 34 35 36 37 38 39 60 61 62 63 64 65 66 67 68 69]


In [54]:
id2.dtype

dtype('int64')